## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import matplotlib.pyplot as plt

# Import API key
from config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,East London,ZA,-33.0153,27.9116,64.18,70,22,15.32,few clouds
1,1,College,US,64.8569,-147.8028,53.22,44,20,1.99,few clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,42.46,65,40,16.11,scattered clouds
3,3,Naze,JP,28.3667,129.4833,63.52,74,96,12.17,light rain
4,4,Stokmarknes,NO,68.5646,14.9108,33.78,80,100,8.05,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
10,10,Roebourne,AU,-20.7833,117.1333,75.22,62,99,10.13,overcast clouds
19,19,Qui Nhon,VN,13.7667,109.2333,77.99,90,100,4.43,overcast clouds
27,27,Rikitea,PF,-23.1203,-134.9692,76.93,74,99,16.91,light rain
34,34,Bengkulu,ID,-3.8004,102.2655,75.45,90,51,4.43,broken clouds
38,38,Novo Aripuana,BR,-5.1206,-60.3797,75.56,95,100,4.72,light rain
43,43,Arraial Do Cabo,BR,-22.9661,-42.0278,76.96,94,40,11.50,scattered clouds
55,55,Hambantota,LK,6.1241,81.1185,77.34,100,20,4.61,few clouds
66,66,Vaini,TO,-21.2000,-175.2000,77.16,94,100,12.66,light rain
70,70,Nzerekore,GN,7.7562,-8.8179,75.49,70,80,1.81,broken clouds
86,86,San Cristobal,VE,7.7669,-72.2250,77.18,77,81,5.10,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().count()

City_ID                67
City                   67
Country                67
Lat                    67
Lng                    67
Max Temp               67
Humidity               67
Cloudiness             67
Wind Speed             67
Current Description    67
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_nonull_df = preferred_cities_df.dropna()
preferred_cities_nonull_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_nonull_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,Roebourne,AU,75.22,overcast clouds,-20.7833,117.1333,
19,Qui Nhon,VN,77.99,overcast clouds,13.7667,109.2333,
27,Rikitea,PF,76.93,light rain,-23.1203,-134.9692,
34,Bengkulu,ID,75.45,broken clouds,-3.8004,102.2655,
38,Novo Aripuana,BR,75.56,light rain,-5.1206,-60.3797,
43,Arraial Do Cabo,BR,76.96,scattered clouds,-22.9661,-42.0278,
55,Hambantota,LK,77.34,few clouds,6.1241,81.1185,
66,Vaini,TO,77.16,light rain,-21.2000,-175.2000,
70,Nzerekore,GN,75.49,broken clouds,7.7562,-8.8179,
86,San Cristobal,VE,77.18,broken clouds,7.7669,-72.2250,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
         print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.75)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))